In [1]:
# Comment: You might need to move this file to the root directory of the project to run it.
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame

from scipy.interpolate import interp1d

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb

from tsfresh.transformers import RelevantFeatureAugmenter, FeatureAugmenter
from tsfresh import extract_relevant_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters

from typing import Dict, Any
import numpy as np
import pandas as pd
from functools import partial

from utils_preprocessing import parse_circuit_params_from_str
from utils_preprocessing import preprocess_data
from utils_preprocessing import unwrap_df

Function definitions:

In [2]:
def parse_circuit_params_from_str(params_str: str) -> Dict[str, float]:
    return {item.split(":")[0].strip(): float(item.split(":")[1].strip()) for item in params_str.split(",")}
    
def process_batch_element_params(Parameters):
    Params = parse_circuit_params_from_str(Parameters)
    return np.array(list(Params.values()))

def process_batch_element_params_str(Parameters):
    Params = parse_circuit_params_from_str(Parameters)
    return np.array(list(Params.keys()))

Process data:

In [3]:
df = preprocess_data("./data/train_data.csv")
df_test = preprocess_data("./data/test_data.csv")

df["param_strs"] = df.apply(lambda x: process_batch_element_params_str(x.Parameters), axis=1)
df["param_values"] = df.apply(lambda x: process_batch_element_params(x.Parameters), axis=1)

df_test["param_strs"] = df_test.apply(lambda x: process_batch_element_params_str(x.Parameters), axis=1)
df_test["param_values"] = df_test.apply(lambda x: process_batch_element_params(x.Parameters), axis=1)


For tsfresh we need to 'unwrap' all the measurements into a dataframe with measurement ids, freq, zreal, and zimag columns

In [4]:
df_ts = unwrap_df(df)
df_ts.shape, len(np.unique(df_ts["id"]))

df_ts_test = unwrap_df(df_test)
df_ts_test.shape, len(np.unique(df_ts_test["id"]))

((55950, 4), 1865)

Train a model for each class of circuit

In [5]:
circuits = np.unique(df["Circuit"])
circuits

array(['L-R-RCPE', 'L-R-RCPE-RCPE', 'L-R-RCPE-RCPE-RCPE', 'RC-G-G',
       'RC-RC-RCPE-RCPE', 'RCPE-RCPE', 'RCPE-RCPE-RCPE',
       'RCPE-RCPE-RCPE-RCPE', 'Rs_Ws'], dtype=object)

In [6]:
dfs = list()
dfs_test = list()
for i in np.arange(len(circuits)):
  dfs.append(df[df["Circuit"] == circuits[i]])
  dfs_test.append(df_test[df_test["Circuit"] == circuits[i]])
dfs[0].head()

,freq,Z,Circuit,Parameters,f,zreal,zimag,param_strs,param_values
9,"[0.1, 0.149213983, 0.222648128, 0.33222214, 0....","[(1237.55544252-0.0137576222j), (1237.55545322...",L-R-RCPE,"L1: 9.04e-07, R1: 2.37e+01, R2: 1.21e+03, CPE1...","[10.0, 13.73823795883263, 18.873918221350976, ...","[1237.5560050720353, 1237.555392001476, 1237.5...","[-1.3822918101198913, -1.8996376035109108, -2....","[L1, R1, R2, CPE1_t, CPE1_C]","[9.04e-07, 23.7, 1210.0, 1.0, 9.36e-08]"
27,"[1.0, 1.34171284, 1.80019333, 2.4153425, 3.240...","[(0.64168195-23.2407201j), (0.57996447-17.3533...",L-R-RCPE,"L1: 2.52e-06, R1: 4.05e-01, R2: 5.45e+04, CPE1...","[10.0, 13.73823795883263, 18.873918221350976, ...","[0.4288148842015722, 0.4223762845566246, 0.417...","[-2.3884593585330647, -1.732490487189707, -1.2...","[L1, R1, R2, CPE1_t, CPE1_C]","[2.52e-06, 0.405, 54500.0, 0.994, 0.043]"
29,"[0.1, 0.149623566, 0.223872114, 0.334965439, 0...","[(3.32454359-3.37348466e-07j), (3.32454358-4.9...",L-R-RCPE,"L1: 2.56e-08, R1: 1.56e+00, R2: 1.76e+00, CPE1...","[10.0, 13.73823795883263, 18.873918221350976, ...","[3.324541232927524, 3.3245403980313397, 3.3245...","[-2.5680336968910618e-05, -3.46355880500947e-0...","[L1, R1, R2, CPE1_t, CPE1_C]","[2.56e-08, 1.56, 1.76, 0.942, 9.61e-07]"
30,"[0.01, 0.0134051824, 0.0179698915, 0.024088967...","[(1089.8535813-181.514229j), (1055.79878508-20...",L-R-RCPE,"L1: 9.74e-07, R1: 3.98e+00, R2: 1.23e+03, CPE1...","[10.0, 13.73823795883263, 18.873918221350976, ...","[33.579076421842096, 27.708840892100422, 23.01...","[-47.45119722165075, -38.68338352830796, -31.4...","[L1, R1, R2, CPE1_t, CPE1_C]","[9.74e-07, 3.98, 1230.0, 0.67, 0.00378]"
32,"[10.0, 14.9249555, 22.2754295, 33.2459793, 49....","[(95766.24176939-0.87437686j), (95766.08903648...",L-R-RCPE,"L1: 1.88e-06, R1: 2.56e+00, R2: 9.58e+04, CPE1...","[10.0, 13.73823795883263, 18.873918221350976, ...","[95766.24176939, 95766.12583901017, 95765.9816...","[-0.87437686, -1.079492885085377, -1.334653446...","[L1, R1, R2, CPE1_t, CPE1_C]","[1.88e-06, 2.56, 95800.0, 0.673, 2.33e-11]"


In [7]:
circuit_str = []
for i in range(len(circuits)):
    for j in range(len(dfs[i]["param_strs"].iloc[0])):
        if j==0: 
            circuit_str.append(circuits[i])
        else: 
            circuit_str.append("")

In [8]:
df_ = dfs[0]
df_y = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])

In [9]:
def ecm_regression(use_dummy=False, quantile=False, relevant=False):
    ppls = list()
    param_values_pred = list(range(df.shape[0]))
    predictions = []
    y_true = []
    circuit_str = []
    for i, [df_, df_test_] in enumerate(zip(dfs, dfs_test)):
        print(" ")
        print("Fitting regression for %s" % df_["Circuit"].loc[df_.index[0]])
        df_ts_ = df_ts[df_ts["id"].isin(df_.index)]
        df_ts_test_ = df_ts_test[df_ts_test["id"].isin(df_test[df_test["Circuit"] == df_["Circuit"].loc[df_.index[0]]].index)]
        X_train = pd.DataFrame(index=np.unique(df_ts_["id"]))
        y_train = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])
        X_test = pd.DataFrame(index=np.unique(df_ts_test_["id"]))
        y_test = pd.DataFrame(df_test_['param_values'].to_list(), columns=df_test_["param_strs"].loc[df_test_.index[0]])
        # transformer mask, tell the transformer which parameters to skip over
        mask = [n for n, x in enumerate(df_["param_strs"].loc[df_.index[0]]) if '_t' in x]
        if not use_dummy:
            mdl = MultiOutputRegressor(estimator=xgb.XGBRegressor(random_state=42))
        else: 
            mdl = MultiOutputRegressor(estimator=DummyRegressor(strategy='median'))

        if quantile:
            regr = TransformedTargetRegressor(regressor=mdl,
                                                transformer=QuantileTransformer(n_quantiles=10, random_state=0))
                                                #transformer=PowerTransformer())
        else:
            regr = mdl

        if relevant:
            ppl = Pipeline([
                    ('augmenter', RelevantFeatureAugmenter(column_id='id', column_sort='freq', default_fc_parameters=ComprehensiveFCParameters())),
                    ('regressor', regr)
                ])
        else:
            ppl = Pipeline([
                    ('augmenter', FeatureAugmenter(column_id='id', column_sort='freq', default_fc_parameters=ComprehensiveFCParameters())),
                    ('regressor', regr)
                ])
        ppl.set_params(augmenter__timeseries_container=df_ts_);
        ppl.fit(X_train, y_train)
        ppl.set_params(augmenter__timeseries_container=df_ts_test_);
        y_pred = ppl.predict(X_test)
        y_pred_df = pd.DataFrame(y_pred, columns=df_test_["param_strs"].loc[df_test_.index[0]])
        y_true_df = pd.DataFrame(y_test, columns=df_test_["param_strs"].loc[df_test_.index[0]])
        predictions.append(y_pred_df)
        y_true.append(y_true_df)
        print("MAE:%.4g" % mean_absolute_error(y_test, y_pred))
        p_strs = df_["param_strs"].loc[df_.index[0]]
        for j in np.arange(len(p_strs)):
            print("%s MAE:%.4g" % (p_strs[j], mean_absolute_error(y_test[p_strs[j]], y_pred[:,j])))
            if j==0: 
                circuit_str.append(circuits[i])
            else: 
                circuit_str.append("")
        ppls.append(ppl)
        #for idx, i in zip(df_.index, np.arange(y_pred.shape[0])):
            #param_values_pred[idx] = y_pred[i,:]
    return predictions, y_true, ppls, circuit_str

In [10]:
predictions_xg, y_true, ppls_xg, circuit_str = ecm_regression(use_dummy=False, quantile=False)
predictions_xgt, y_true, ppls_xgt, circuit_str = ecm_regression(use_dummy=False, quantile=True)
predictions_d, y_true, ppls_d, circuit_str = ecm_regression(use_dummy=True, quantile=False)

 
Fitting regression for L-R-RCPE


Feature Extraction: 100%|██████████| 25/25 [00:02<00:00,  8.67it/s]


MAE:7762
L1 MAE:1.812e-06
R1 MAE:36.18
R2 MAE:3.877e+04
CPE1_t MAE:0.01571
CPE1_C MAE:0.003598
 
Fitting regression for L-R-RCPE-RCPE


Feature Extraction: 100%|██████████| 25/25 [00:02<00:00,  9.04it/s]


MAE:6746
L1 MAE:1.787e-06
R1 MAE:45.37
R2 MAE:322.5
CPE1_t MAE:0.1273
CPE1_C MAE:3.569e-06
R3 MAE:5.36e+04
CPE2_t MAE:0.02882
CPE2_C MAE:0.007694
 
Fitting regression for L-R-RCPE-RCPE-RCPE


...

MAE:8196
R1 MAE:11.95
R2 MAE:328.6
R3 MAE:7.343e+04
CPE1_t MAE:0.1251
CPE1_C MAE:1.741e-06
CPE2_t MAE:0.1273
CPE2_C MAE:5.95e-05
CPE3_t MAE:0.1226
CPE3_C MAE:0.02729
 
Fitting regression for RCPE-RCPE-RCPE-RCPE


Feature Extraction: 100%|██████████| 25/25 [00:12<00:00,  2.04it/s]
/Users/joachim/opt/anaconda3/envs/eis-battmen/lib/python3.7/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: `n_features_in_` is deprecated in 1.0 and will be removed in 1.2.
  warnings.warn(msg, category=FutureWarning)
Feature Extraction: 100%|██████████| 25/25 [00:03<00:00,  8.31it/s]


MAE:7764
R1 MAE:39.29
R2 MAE:137.5
R3 MAE:2503
R4 MAE:9.049e+04
CPE1_t MAE:0.1298
CPE1_C MAE:4.552e-07
CPE2_t MAE:0.1308
CPE2_C MAE:0.0008804
CPE3_t MAE:0.1294
CPE3_C MAE:0.003501
CPE4_t MAE:0.1297
CPE4_C MAE:0.02773
 
Fitting regression for Rs_Ws


Feature Extraction: 100%|██████████| 25/25 [00:04<00:00,  5.93it/s]
/Users/joachim/opt/anaconda3/envs/eis-battmen/lib/python3.7/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: `n_features_in_` is deprecated in 1.0 and will be removed in 1.2.
  warnings.warn(msg, category=FutureWarning)
Feature Extraction: 100%|██████████| 24/24 [00:01<00:00, 21.06it/s]


MAE:268.2
R1 MAE:100.4
W1_R MAE:783
W1_T MAE:188.6
W1_p MAE:0.9692


In [11]:
def MAE_loss(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def parse_mae(predictions, y_true, circuit_str, model_names, save=False, save_path=None):
    results = []
    k = 0
    for i in range(len(predictions[0])):
        for j, col in enumerate(predictions[0][i].columns):
            mae_test_element = []
            for prediction in predictions:
                mae_test_element.append(MAE_loss(y_true[i][col], prediction[i][col]))
            res_element = [circuit_str[k], col] + mae_test_element
            results.append(res_element)
            k += 1
    if save:
        # Save results to csv
        results_df = pd.DataFrame(results, columns=['Circuit', 'Parameter'] + model_names)
        results_df.to_csv(save_path+'results.csv', index=False)
    return results, results_df

In [12]:
model_names = ['Dummy', 'tsfresh-XGBoost', 'tsfresh-XGBoost-Quantile']
results_xg, results_df = parse_mae([predictions_d, predictions_xg, predictions_xgt], y_true, circuit_str, model_names, save=True, save_path='reg-')

In [13]:
results_df.to_latex('reg-results.tex', index=False)